In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
readIn = pd.read_csv("data/df_grouped_by_borrower.csv")
df = pd.DataFrame(readIn)


In [3]:
readIn = pd.read_csv("data/data_submission_example.csv")
df_target = pd.DataFrame(readIn)
# df_target.head(20)

# print(df_target.shape)
# print(df_target['BORROWER_ID'].unique)

In [4]:
df

,BORROWER_ID,target,CONTRACT_ID_SUM,CONTRACT_CREDIT_LOSS_SUM,CONTRACT_DEPT_SERVICE_TO_INCOME_MEAN,CONTRACT_INCOME_SUM,CONTRACT_INSTALMENT_AMOUNT_MEAN,CONTRACT_INTEREST_RATE_MEAN,CONTRACT_LGD_SUM,CONTRACT_LOAN_AMOUNT_SUM,...,CONTRACT_REFINANCED_10001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_0.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100003.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100004.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_110001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140003.0
0,d8SqtuEV,0,3,13831.0,27.76,256244.0,27149.0,17.31,3295461.0,3367556,...,0,2,0,1,0,0,0,0,0,0
1,gII7nnq4,0,1,16350.0,7.05,127305.0,8899.0,22.35,282553.0,275924,...,0,0,0,0,1,0,0,0,0,0
2,MMkJ8z/e,0,1,0.0,0.00,0.0,14329.0,0.00,68981.0,138862,...,0,1,0,0,0,0,0,0,0,0
3,M417onFP,0,1,2395.0,0.00,0.0,4003.0,23.92,163319.0,169624,...,0,0,0,1,0,0,0,0,0,0
4,u9GRAuen,0,1,0.0,0.00,0.0,11214.0,35.12,74244.0,100079,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135553,5IwSjcnC,0,1,8068.0,39.36,177504.0,21284.0,12.47,1495241.0,996664,...,0,0,0,0,0,1,0,0,0,0
1135554,FWvYyvQ0,0,1,8068.0,39.36,177504.0,21284.0,12.47,1495241.0,996664,...,0,0,0,0,0,1,0,0,0,0
1135555,oJ6WMUhN,0,1,79782.0,46.82,183837.0,37976.0,18.64,1833694.0,1943249,...,0,0,0,0,1,0,0,0,0,0
1135556,8rDUSdWG,0,1,7506.0,25.08,108478.0,12793.0,20.57,639540.0,344105,...,0,0,0,0,0,1,0,0,0,0


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Bemenő változók és célváltozó meghatározása
bemenő_változók = df.columns.drop(['BORROWER_ID', 'target'])  # Az összes oszlop, kivéve a 'BORROWER_ID' és 'target'
X = df[bemenő_változók]  # Bemenő változók
y = df['target']  # Célváltozó

# Modell inicializálása és illesztése az adatokra
gbm = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=22)
gbm.fit(X, y)

GradientBoostingClassifier(n_estimators=50, random_state=22)

# SAVING RESULTS

In [6]:
merged_df = df[df['BORROWER_ID'].isin(df_target['BORROWER_ID'])]

In [7]:
X = merged_df[bemenő_változók]

In [8]:
merged_df['predicted_target'] = gbm.predict_proba(X)[:,1]

C:\Users\User\AppData\Local\Temp\ipykernel_17768\260587743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['predicted_target'] = gbm.predict_proba(X)[:,1]


In [9]:
merged_df2=pd.merge(df_target,merged_df, on='BORROWER_ID', how='left')

In [10]:
cols=['BORROWER_ID', 'predicted_target', 'PRED']

res=merged_df2[cols]

In [11]:
res['predicted_target'].isna().sum()

13674

In [12]:
res['predicted_target'] = res['predicted_target'].fillna(res['PRED'])


C:\Users\User\AppData\Local\Temp\ipykernel_17768\4216634701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['predicted_target'] = res['predicted_target'].fillna(res['PRED'])


In [13]:
res.head(10)


,BORROWER_ID,predicted_target,PRED
0,+++b/oYx,0.000567,0.014477
1,+++pcjqH,0.000567,0.014477
2,+++ukYrT,0.000567,0.014477
3,+++wvYn7,0.000567,0.014477
4,++/6JzSJ,0.000621,0.014477
5,++/Ys2ls,0.012924,0.014477
6,++/faYdJ,0.000950,0.014477
7,++/rT+Iv,0.000567,0.014477
8,++0Mv31Z,0.000567,0.014477
9,++0yhZwt,0.000567,0.014477


In [14]:
res=res.drop('PRED', axis=1)

In [15]:
res.columns=['BORROWER_ID', 'PRED']

In [16]:
# multi = 0.0148/res['PRED'].mean()
# res['PRED']=res['PRED']*multi
res['PRED'].mean()

0.004611756498964255

In [17]:
print(res.shape)
res.to_csv('data/results.csv', index=False)

(1117674, 2)


In [18]:
df_target.shape

(1117674, 2)

In [21]:
res['PRED'].mean()

0.004611756498964255